In [1]:
#ライブラリ
import tweepy
import nltk
import stanza
import pandas as pd
import numpy as np
import re
import time
import schedule
import datetime

In [2]:
#Twitter API config読み込み
config = pd.read_csv("./config.csv")

In [3]:
# Twitter API config
twitterApiKey = config['twitterApiKey'][0]
twitterApiSecret = config['twitterApiSecret'][0]
twitterApiAccessToken = config['twitterApiAccessToken'][0]
twitterApiAccessTokenSecret = config['twitterApiAccessTokenSecret'][0]

In [4]:
# Authenticate
auth = tweepy.OAuthHandler(twitterApiKey, twitterApiSecret)
auth.set_access_token(twitterApiAccessToken, twitterApiAccessTokenSecret)
api = tweepy.API(auth, wait_on_rate_limit = True)

In [5]:
# ツイート収集&出力タスク定義
def job():
    #検索ワード、条件
    s='btc min_faves:3 -filter:retweets'
    
    #5日前と4日前の時刻を取得
    dt  = datetime.date.today()
    dt5 = dt + datetime.timedelta(days=-5)
    dt4 = dt + datetime.timedelta(days=-4)

    #ツイート収集
    tweets = tweepy.Cursor(api.search,  
                           q=s,
                           exclude_replies=True,
                           tweet_mode='extended',
                           lang = 'en',
                           since= dt5,
                           until= dt4,
                          ).items(10000);
    
    #ツイートのリスト
    t_data = []
    for tweet in tweets:
        t_data.append([tweet.user.name,
                       '@'+tweet.user.screen_name,
                       tweet.user.friends_count,
                       tweet.user.followers_count,
                       tweet.full_text.replace('\n',''),
                       tweet.favorite_count,
                       tweet.retweet_count,
                       tweet.created_at])


              
    #処理前ツイート出力
    df = pd.DataFrame(data=t_data, columns=['Name','User Name','Following','Followers','Tweet','Favorite','RT','created at'])
    
    df.to_csv(f'./tweet-of-btc/{dt5} ~ {dt4}.csv')

    #件数確認
    dt_now = datetime.datetime.now()
    print('現在時刻:{}\n{} ~ {}：{}件収集完了\n\n'.format(dt_now,dt5,dt4,len(t_data)))
    
    #ログファイルに記録
    df2 = pd.read_csv('./tweet-of-btc/assembling-log.csv')
    s = pd.Series([dt_now,dt5,dt4,len(t_data)], index=['現在時刻', '区間(start)','区間(end)','収集件数'])
    df2 = df2.append(s, ignore_index=True)
    df2.to_csv('./tweet-of-btc/assembling-log.csv')

In [6]:
# 定時実行
def main():
    schedule.every().day.at("12:00").do(job)

    while True:
        schedule.run_pending()
        time.sleep(1)

In [ ]:
# main関数実行
if __name__ == "__main__":
    main()

現在時刻:2021-08-06 12:30:29.606516
2021-08-01 ~ 2021-08-02：5878件収集完了


現在時刻:2021-08-07 12:30:32.039524
2021-08-02 ~ 2021-08-03：5973件収集完了


現在時刻:2021-08-08 12:30:10.393740
2021-08-03 ~ 2021-08-04：5394件収集完了


現在時刻:2021-08-09 12:30:22.498061
2021-08-04 ~ 2021-08-05：5705件収集完了


現在時刻:2021-08-10 12:30:30.140484
2021-08-05 ~ 2021-08-06：5947件収集完了


現在時刻:2021-08-11 12:30:47.710744
2021-08-06 ~ 2021-08-07：6469件収集完了


